# ML Hands-on Challenge - Getting started 

The goal of this notebook is to explore the data that we have provided from the ML hands-on challenge. You will learn more about the CATH labels, how to visualize the protein structure, and challenges you will have to handle (e.g. gaps in structure). 

## 0. Setup

In [ ]:
!pip install biopython
!pip install py3dmol

In [ ]:
import pandas as pd
import numpy as np
import sys
import glob
import os
import Bio.PDB.PDBParser
import py3Dmol
import warnings

sys.path.append('../')

warnings.filterwarnings("ignore", message="Used element '.' for Atom")

architecture_names = {(1, 10): "Mainly Alpha: Orthogonal Bundle",
                      (1, 20): "Mainly Alpha: Up-down Bundle",
                      (2, 30): "Mainly Beta: Roll",
                      (2, 40): "Mainly Beta: Beta Barrel",
                      (2, 60): "Mainly Beta: Sandwich",
                      (3, 10): "Alpha Beta: Roll",
                      (3, 20): "Alpha Beta: Alpha-Beta Barrel",
                      (3, 30): "Alpha Beta: 2-Layer Sandwich",
                      (3, 40): "Alpha Beta: 3-Layer(aba) Sandwich",
                      (3, 90): "Alpha Beta: Alpha-Beta Complex"}

## 1. Opening the data

In [ ]:
# Open the training data sequences and structure
data = pd.read_csv('../data/cath_w_seqs_share.csv', index_col=0)
data

In [ ]:
# The CATH classification for a protein can be determined by concatenating the columns
example_cath_id = data['cath_id'][0]
example_class = data['class'][0]
example_arch = f"({example_class},{data['architecture'][0]})"
example_topo = f"({example_class},{data['architecture'][0]},{data['topology'][0]})"
example_superfam = f"({example_class},{data['architecture'][0]},{data['topology'][0]},{data['superfamily'][0]})"

print(f"""
Protein domain with cath id {example_cath_id} is in class {example_class}, \
architecture {example_arch}, topology {example_topo}, and superfamily {example_superfam}.
""")

For this example cath id "2w3sB01," your model should ideally classify it as architecture (3,90). Also, from this example, you can deduce how to construct the labels that you need.

In [ ]:
# NOTE(milo): Couldn't get this command to run on a Git LFS pointer file.
!unzip pdb_share.zip

In [ ]:
# The sequences come from the PDB files
from Bio.PDB.Polypeptide import protein_letters_3to1

def get_sequence_from_pdb(pdb_filename):
    pdb_parser = Bio.PDB.PDBParser()
    structure = pdb_parser.get_structure(pdb_filename, pdb_filename)
    assert len(structure) == 1

    seq = []

    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_id()[0] == " ":  # This checks if it's a standard residue
                    seq.append(protein_letters_3to1[residue.get_resname()])
                else:
                    print('nonstandard', residue.get_id())

    return ''.join(seq)

example_seq = get_sequence_from_pdb(f"../data/pdb_share/{example_cath_id}")
print(f"The sequence for cath id {example_cath_id} is {example_seq}")

# Check that it matches the data file
data['sequences'][0] == example_seq

## 2. Visualize some examples

We are grouping at the CATH architecture level as this is the level that your models will be classifying the protein domains.

There are 10 CATH architectures that the protein domains can be classified into, and we will explore examples from each architecture.

In [ ]:
# Load sequence and structure for one example for each architecture
cath_examples = data.groupby(['class', 'architecture'])[['cath_id','sequences']].first()
cath_examples

In [ ]:
def view_structure(pdb_filename, name, gaps=[], width=300, height=300):
  pdb_parser = Bio.PDB.PDBParser()
  structure = pdb_parser.get_structure(pdb_filename, pdb_filename)

  # Add the model and set the cartoon style
  viewer = py3Dmol.view(query=f'arch: {name}, pdb: {pdb_filename}', width=width, height=height)
  viewer.addModel(open(pdb_filename, 'r').read(), 'pdb')
  viewer.setStyle({'cartoon': {'color': 'spectrum'}})

  if gaps:
    # Add dashed lines for gaps
    for chain_id, start_res, end_res in gaps:
        try:
            start_residue = structure[0][chain_id][start_res-1]
            end_residue = structure[0][chain_id][end_res]

            # Get coordinates and convert to Python float
            start_coords = [float(coord) for coord in start_residue['CA'].get_coord()]
            end_coords = [float(coord) for coord in end_residue['CA'].get_coord()]

            # Add dashed cylinders for missing residues
            viewer.addCylinder({'start': {'x': start_coords[0], 'y': start_coords[1], 'z': start_coords[2]},
                                'end': {'x': end_coords[0], 'y': end_coords[1], 'z': end_coords[2]},
                                'radius': 0.1, 'color': 'red', 'dashed': True, 'fromCap': 1, 'toCap': 1})
        except KeyError:
          print(f"Residue {start_res} or {end_res} in chain {chain_id} not found.")

  viewer.zoomTo()
  return viewer

In [ ]:
import py3Dmol
from IPython.display import display, HTML

# NOTE(milo): Modified this file path!
pdb_dir = '../data/pdb_share'
num_columns = [2, 3, 5]  # Number of columns in the grid
# titles = ['Structure 1', 'Structure 2', 'Structure 3', 'Structure 4']

# Creating HTML table for the grid
html = '<table><tr>'
print_col = 0
for i, (arch, row) in enumerate(cath_examples.iterrows()):
    cath_id = row[0]
    pdb_filename = os.path.join(pdb_dir, cath_id)

    if (i-sum(num_columns[:print_col])) % num_columns[print_col] == 0 and i > 0:
        print_col += 1
        html += '</tr><tr>'
    viewer_html = view_structure(pdb_filename, arch)._make_html()
    html += f'<td><div style="text-align: center;"><strong>{arch} {architecture_names[arch]}</strong></div>{viewer_html}</td>'
html += '</tr></table>'

# Display the grid
display(HTML(html))


## 3. Note the gaps

Since the protein domain structures are experimental determined, there are some regions of the domain that failed to resolve. These residue show up as gaps in the protein domain sequence and structure.

In [ ]:
# Indices of the cath domain associated with the cath_id in the full protein
# that can be found using the pdb id in the PDB online.

example_cath_ids = ['3zq4C03', '3i9v600']
tmp = data[data['cath_id'].isin(example_cath_ids)]

print(tmp.iloc[0])
print(len(tmp.iloc[0].sequences))
tmp.iloc[0].cath_indices
print(555 - 449)

In [ ]:
# If you compare the indices that in the cath_indices range and not in the PDB file
# residue indices, for these examples you will get this
# NOTE(milo): The cath_indicates are the relevant parts of PDB file for the protein domain.
# NOTE(milo): The gap indices below are not inclusive of the ending index.
example_gaps = {'3zq4C03': [('C', 493, 501)],
                '3i9v600': [('6', 58, 74)]}

In [ ]:
# We can visualize the gaps are red lines
import py3Dmol
from IPython.display import display, HTML

# Creating HTML table for the grid
html = '<table><tr>'
for cath_id, gap in example_gaps.items():
  viewer_html = view_structure(f'pdb_share/{cath_id}', cath_id, gaps=gap)._make_html()
  html += f'<td><div style="text-align: center;"><strong>{cath_id}</strong></div>{viewer_html}</td>'
html += '</tr></table>'

# Display the grid
display(HTML(html))

## Understanding the PDB files

In [ ]:
# Using 1a0sP00 as an example
row = data[data.cath_id == '1a0sP00'].iloc[0]

print("The sequence of amino acids is:", row.sequences)

In [ ]:
# This is some example data from the PDB file:
# ATOM      1  N   SER P  71     -47.333   0.941   8.834  1.00 52.56
# ATOM      2  CA  SER P  71     -45.849   0.731   8.796  1.00 53.56
# ATOM      3  C   SER P  71     -45.191   1.608   7.714  1.00 51.61
# ATOM      4  O   SER P  71     -45.455   2.818   7.648  1.00 54.49
# ATOM      5  CB  SER P  71     -45.511  -0.764   8.600  1.00 55.68
# ATOM      6  OG  SER P  71     -46.116  -1.305   7.434  1.00 58.53
# ATOM      7  N   GLY P  72     -44.347   1.018   6.868  1.00 46.18
# ATOM      8  CA  GLY P  72     -43.702   1.805   5.836  1.00 38.59
# ATOM      9  C   GLY P  72     -43.533   1.109   4.498  1.00 34.81
# ATOM     10  O   GLY P  72     -44.500   0.739   3.827  1.00 32.75

# According to this resource: https://www.cgl.ucsf.edu/chimera/docs/UsersGuide/tutorials/pdbintro.html
# The atom names have
# - the name of the atom (like N for nitrogen)
# - something called the remoteness indicator code (A, B, G, D, E, Z, H)
# - a branch indicator, if required

In [ ]:
pdb_filename = "../data/pdb_share/1a0gA02"
pdb_parser = Bio.PDB.PDBParser()
structure = pdb_parser.get_structure(pdb_filename, pdb_filename)

# Add the model and set the cartoon style
name = "TMP"
width = 300
height = 300
viewer = py3Dmol.view(query=f'arch: {name}, pdb: {pdb_filename}', width=width, height=height)
viewer.addModel(open(pdb_filename, 'r').read(), 'pdb')
viewer.setStyle({'cartoon': {'color': 'spectrum'}})
viewer.zoomTo()
dir(viewer.getModel())

In [ ]:
import pywavefront
scene = pywavefront.Wavefront("../data/obj/16pkA02.obj")

In [ ]:
scene